<a href="https://colab.research.google.com/github/Wilkne-Maia/Portfolio/blob/main/Analisando_Dados_da_Empresa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Analisar uma série temporal de consumo de eletricidade: com python e pandas

O objetivo deste exemplo será aplicar tarifas de energia por tempo de uso para encontrar o custo total do consumo de energia por um ano. Ou seja, em diferentes horas do dia, o preço da eletricidade varia, então a tarefa é multiplicar a eletricidade consumida em cada hora pelo preço correto para a hora em que foi consumida.

In [38]:
# importar os pacotes necessários
import pandas as pd

# verificar versão do pandas
pd.__version__

'1.4.3'

> Vamos ler nossos dados de um arquivo CSV que possui duas colunas: uma para data mais hora e outra para energia elétrica consumida em quilowatts-hora (kWh)

In [23]:
# Certifique-se de que `demand_profile.csv` esteja em seu diretório de trabalho atual.
df = pd.read_csv('demand_profile.csv')
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


> Pandas e NumPy têm um conceito de dtypes(tipos de dados). Se nenhum argumento for especificado, date_time assumirá um tipo `object`:

In [24]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

> Para trabalhar com dados de séries temporais, você desejará que a  coluna date_time seja formatada como uma matriz de objetos de data e hora. (O Pandas chama isso de Timestamp.).

In [25]:
df['date_time'] = pd.to_datetime(df['date_time'])

df.dtypes
# type(df.iat[0, 0])

date_time     datetime64[ns]
energy_kwh           float64
dtype: object

 > Vamos usar um decorador de temporização , que chamei de forma não original @timeit. Esse decorador imita amplamente timeit.repeat() a biblioteca padrão do Python, mas permite que você retorne o resultado da própria função e imprima seu tempo de execução médio de várias tentativas. (O Python timeit.repeat()retorna os resultados de tempo, não o resultado da função.)

In [26]:
# função para calcular o tempo de execução de uma função
import functools
import gc
import itertools
import sys
from timeit import default_timer as _timer

def timeit(_func=None, *, repeat=3, number=1000, file=sys.stdout):
    _repeat = functools.partial(itertools.repeat, None)

    def wrap(func):
        @functools.wraps(func)
        def _timeit(*args, **kwargs):
            gcold = gc.isenabled()
            gc.disable()

            try:
                trials = []
                for _ in _repeat(repeat):
                    total = 0
                    for _ in _repeat(number):
                        start = _timer()
                        result = func(*args, **kwargs)
                        end = _timer()
                        total += end - start
                    trials.append(total)
                best = min(trials) / number
                print(
                    "Best of {} trials with {} function"
                    " calls per trial:".format(repeat, number)
                )
                print(
                    "Function `{}` ran in average"
                    " of {:0.3f} seconds.".format(func.__name__, best),
                    end="\n\n",
                    file=file,
                )
            finally:
                if gcold:
                    gc.enable()
            return result

        return _timeit

    if _func is None:
        return wrap
    else:
        return wrap(_func)

In [27]:
@timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

# Leia novamente para que tenhamos o dtype `object` para iniciar
df['date_time'] = convert(df, 'date_time')

Best of 3 trials with 10 function calls per trial:
Function `convert` ran in average of 0.002 seconds.



> O Pandas será muito mais rápido quanto menos precisar interpretar seus dados. Nesse caso, você verá grandes melhorias de velocidade apenas informando ao Pandas como são seus dados de data e hora, usando o parâmetro de formato.

In [28]:
@timeit(repeat=3, number=100)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name], format='%d/%m/%y %H:%M')

> Os datetimes no CSV não estão no formato ISO 8601 : você precisaria de YYYY-MM-DD HH:MM. Se você não especificar um formato, o Pandas usará o **[dateutil](https://dateutil.readthedocs.io/en/stable/])** pacote para converter cada string em uma data.

## Looping simples sobre dados do Pandas

> Agora que suas datas e horários estão em um formato conveniente, você está pronto para começar a calcular seus custos de eletricidade. Lembre-se de que o custo varia de acordo com a hora, portanto, você precisará aplicar condicionalmente um fator de custo a cada hora do dia. Neste exemplo, os custos de tempo de uso serão definidos da seguinte forma:
> 
Tipo de tarifa | Centavos por kWh | Período
:--------- | :------: | -------:
peak_hours | 28 | 17h00 às 24h00
shoulder_hours | 20 | 7:00 às 17:00
off_peak_hours | 12 | 0:00 às 7:00


> Se o preço fosse fixo de 28 centavos por kWh para cada hora do dia, esse cálculo poderia ser feito em uma linha.<br>
> Resultará na criação de uma nova coluna com o custo da eletricidade para aquela hora.

In [195]:
df['cost_cents'] = df['energy_kwh'] * 28

df.head()

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,16.408
1,2013-01-01 01:00:00,0.580,16.240
2,2013-01-01 02:00:00,0.572,16.016
3,2013-01-01 03:00:00,0.596,16.688
4,2013-01-01 04:00:00,0.592,16.576


## Vejamos uma abordagem de `loop`

In [13]:
# NOTA: Não faça isso!
# def apply_tariff(kwh,   hour):
#     """Calculates cost of electricity for given hour."""    
#     if 0 <= hour < 7:
#         rate = 12
#     elif 7 <= hour < 17:
#         rate = 20
#     elif 17 <= hour < 24:
#         rate = 28
#     else:
#         raise ValueError(f'Invalid hour: {hour}')
#     return rate * kwh

In [ ]:
# NOTA: Não faça isso!
# @timeit(repeat=3, number=100)
# def apply_tariff_loop(df):
#     """Calcule os custos em loop. Modifica o `df` no local."""
#     energy_cost_list = []    
#     for i in range(len(df)):
#         # Obter eletricidade usada e hora do dia
#         energy_used = df.iloc[i]['energy_kwh']
#         hour = df.iloc[i]['date_time'].hour
#         energy_cost = apply_tariff(energy_used, hour)
#         energy_cost_list.append(energy_cost)
#     df['cost_cents'] = energy_cost_list
        
# apply_tariff_loop(df)

In [ ]:
# NOTA: Não faça isso!
# @timeit(repeat=3, number=100)
# def apply_tariff_iterrows(df):
#     energy_cost_list = []
#     for index, row in df.iterrows():
#         # Obter eletricidade usada e hora do dia
#         energy_used = row['energy_kwh']
#         hour = row['date_time'].hour
#         # Anexar lista de custos
#         energy_cost = apply_tariff(energy_used, hour)
#         energy_cost_list.append(energy_cost)
#     df['cost_cents'] = energy_cost_list
    
# apply_tariff_iterrows(df)

## Pandas `.apply()`

> O método do Pandas .apply() pega funções (callables) e as aplica ao longo de um eixo de um DataFrame (todas as linhas ou todas as colunas). Neste exemplo, uma função **[lambda]()** ajudará você a passar as duas colunas de dados para apply_tariff():

In [15]:
@timeit(repeat=3, number=100)
def apply_tariff_withapply(df):
    df['cost_cents'] = df.apply(
        lambda row: apply_tariff(
            kwh=row['energy_kwh'],
            hour=row['date_time'].hour
        ),
        axis=1
    )
apply_tariff_withapply(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_withapply` ran in average of 0.042 seconds.



## Selecionando dados com `.isin()`

> Você verá como selecionar linhas com o `.isin()` método do Pandas e então aplicar a tarifa adequada em uma operação vetorizada. Antes de fazer isso, as coisas ficarão um pouco mais convenientes se você definir a date_timecoluna como o índice do DataFrame:

In [29]:
df.set_index('date_time', inplace=True)

@timeit(repeat=3, number=100)
def apply_tariff_isin(df):
    # Define hour range Boolean arrays
    peak_hours = df.index.hour.isin(range(17, 24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    off_peak_hours = df.index.hour.isin(range(0, 7))

    # Apply tariffs to hour ranges
    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12

> Vamos ver como isso se compara:

In [30]:
apply_tariff_isin(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_isin` ran in average of 0.001 seconds.



> Para entender o que está acontecendo neste código, você precisa saber que o método `.isin()` está retornando uma matriz de valores booleanos que se parece com isso:
>
>>~~~python
>>[False, False, False, ..., True, True, True]
>>~~~

> Esses valores identificam quais índices DataFrame (datetimes) estão dentro do intervalo de horas especificado. Então, quando você passa essas matrizes booleanas para o .locindexador do DataFrame, obtém uma fatia do DataFrame que inclui apenas as linhas que correspondem a essas horas. Depois disso, é só multiplicar a fatia pela tarifa adequada, que é uma operação vetorizada veloz.

## Podemos fazer melhor?

> você pode fazer as coisas ainda mais programaticamente com a função `pd.cut()` do Pandas neste caso:

In [31]:
@timeit(repeat=3, number=100)
def apply_tariff_cut(df):
    cents_per_kwh = pd.cut(x=df.index.hour,
                           bins=[0, 7, 17, 24],
                           include_lowest=True,
                           labels=[12, 20, 28]).astype(int)
    df['cost_cents'] = cents_per_kwh * df['energy_kwh']


> O `pd.cut()` está aplicando um array de rótulos (nossos custos) de acordo com qual bin cada hora pertence. Observe que o parâmetro `include_lowest`  indica se o primeiro intervalo deve ser inclusivo à esquerda ou não. (Você deseja incluir time=0 em um grupo.)

In [32]:
apply_tariff_cut(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_cut` ran in average of 0.001 seconds.



## Não se esqueça do NumPy!

> Neste próximo caso, você usará a função `digitize()` do NumPy. É semelhante ao Pandas, `cut()` pois os dados serão agrupados, mas desta vez serão representados por uma matriz de índices representando a qual bin cada hora pertence. Esses índices são então aplicados a uma matriz de preços:

In [39]:
#  importar numpy
import numpy as np

In [40]:
@timeit(repeat=3, number=100)
def apply_tariff_digitize(df):
    prices = np.array([12, 20, 28])
    bins = np.digitize(df.index.hour.values, bins=[7, 17, 24])
    df['cost_cents'] = prices[bins] * df['energy_kwh'].values


In [41]:
apply_tariff_digitize(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_digitize` ran in average of 0.000 seconds.



> Aqui está a “ordem de precedência” acima em ação, com cada função que você criou aqui:
> 
Função                  | Tempo de execução (segundos)
:---------                | :------ 
apply_tariff_loop()       | 3.152
apply_tariff_loop()       | 0.713
apply_tariff_withapply()  | 0.272
apply_tariff_isin()       | 0.010
apply_tariff_cut()        | 0.003
apply_tariff_digitize()   | 0.002


## Conclusões

1. Use operações vetorizadas: métodos e funções Pandas sem `loops for`.
2. Use o método `.apply()` com um callable.
3. Use `.itertuples()`: itere sobre as linhas do DataFrame a partir do módulo `namedtuples` do Python `.collections`.
4. Use `.iterrows()`: itere sobre as linhas do DataFrame como `pd.Seriespares` (index, ). Embora uma série Pandas seja uma estrutura de dados flexível, pode ser caro construir cada linha em uma série e depois acessá-la.
5. Use “elemento por elemento” para loops, atualizando cada célula ou linha uma de cada vez com `df.locou`, `df.iloc`. (Ou .at/ .iatpara acesso escalar rápido.).

Os exemplos que você explorou aqui são bastante diretos, mas ilustram como a aplicação adequada dos recursos do Pandas pode fazer grandes melhorias no tempo de execução e na legibilidade do código para inicializar. Aqui estão algumas regras básicas que você pode aplicar na próxima vez que estiver trabalhando com grandes conjuntos de dados no Pandas:

* Tente usar operações vetorizadas sempre que possível, em vez de abordar problemas com a mentalidade `for x in df...`. Se o seu código abriga muitos `loops for`, ele pode ser mais adequado para trabalhar com estruturas de dados nativas do Python, porque o Pandas vem com muita sobrecarga.

* Se você tiver operações mais complexas em que a vetorização é simplesmente impossível ou muito difícil de trabalhar com eficiência, use o método `.apply()`.

* Se você precisar fazer um `loop` sobre sua matriz (o que acontece), use `.iterrows()` ou `.itertuples()` para melhorar a velocidade e a sintaxe.

* O Pandas tem muitas opções e quase sempre há várias maneiras de ir de A a B. Esteja atento a isso, compare o desempenho de diferentes rotas e escolha a que funciona melhor no contexto do seu projeto.

* Depois de criar um script de limpeza de dados, evite o reprocessamento armazenando seus resultados intermediários com o `HDFStore`.

* A integração do `NumPy` nas operações do Pandas geralmente pode melhorar a velocidade e simplificar a sintaxe.

* **[lambReal Pythonda](https://realpython.com/fast-flexible-pandas/#conclusions)**
* **[Funão timeit()](https://github.com/realpython/materials/blob/master/pandas-fast-flexible-intuitive/tutorial/timer.py)**
* **[GitHub](https://github.com/realpython/materials/tree/master/pandas-fast-flexible-intuitive)**